HW 2 Мельчук А.Б.

In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420317 sha256=40607aa33241e35a7b63ee27163484f15919ed49aca24ac02fdc9130793b6821
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k



In [75]:
data = pd.read_csv('/content/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [76]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [77]:
import random
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    recs = random.choices(items_weights['item_id'], weights=items_weights['weight'], k=n)

    # return recs.tolist()
    return recs

In [112]:
%%time

# your_code

popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity = popularity.rename(columns={"sales_value": "weight"})

popularity['weight'] = popularity['weight'].apply(lambda x: np.log(x + 1e-7))

mn = popularity['weight'].min()
mm =  popularity['weight'].max() - mn
popularity['weight'] = popularity['weight'].apply(lambda x: ((x - mn) / mm) ) 


result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity, n=5))


CPU times: user 18.6 s, sys: 79.8 ms, total: 18.7 s
Wall time: 18.7 s


In [113]:
popularity['weight'].max(),popularity['weight'].min()

(1.0, 0.0)

In [114]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[6442809, 953599, 1000736, 1005614, 7440979]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[7168821, 9878849, 9219945, 1029902, 997025]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [198]:
result = pd.read_csv('/content/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [343]:
# your_code

%precision %.10f
import ast

def precision_at_k(data_, dest_name, k=5):    
    s = 0
    for index, row in data_.iterrows():     
        actual_list = np.array(list( map(int, row['actual'][1:-1].split()) ))
        rec_list = np.array(ast.literal_eval(row[dest_name]))[:k]      
        flags = np.isin(actual_list, rec_list)      
        s += flags.sum() / len(rec_list)            
    return s/data_.shape[0]

In [344]:
precision_at_k(result, 'random_recommendation')

0.0005876592

In [345]:
precision_at_k(result, 'popular_recommendation')

0.1552399608

In [346]:
precision_at_k(result, 'itemitem')

0.0335945152

In [347]:
precision_at_k(result, 'cosine')

0.0352595495

In [348]:
precision_at_k(result, 'tfidf')

0.0361410382

In [349]:
precision_at_k(result, 'own_purchases')

0.1799869409

popular_recommendation лучший

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code